In [1]:
print("hello")

hello


# 📘 Sentiment Analysis App

## 🧩 Introduction

Sentiment analysis is a widely used feature in Natural Language Processing (NLP). It allows us to determine whether a piece of text, such as a product review, expresses a positive, negative, or neutral sentiment.

However, our app will go beyond basic sentiment detection. It will be a **text classification application**, also known as a **tagging app**. The goal is to assign labels to text based on its content.

Our app will be able to classify text into multiple categories, such as:

- **Sentiment**: Detect the sentiment of the text (positive, negative, neutral).
- **Language**: Identify the language in which the text is written.
- **Style**: Analyze writing style (formal, informal, etc.).
- **Topics or Categories**: Detect the main subject or theme (e.g., sports, politics, technology).
- **Political Tendency**: Analyze the political leaning of the text.
- **And more...**

This flexibility makes our app a powerful tool for a variety of NLP use cases.

In [2]:
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from dotenv import load_dotenv, find_dotenv

In [3]:
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model= "llama-3.3-70b-versatile",
    temperature= 0.1
)

## 🏷️ Tag Definition

Instead of using the previous language model, we will define a new LLM in the following code block and utilize the `with_structured_output` method supported by OpenAI models.

### 🔖 Tags Used in This App

In the following section, we define the three tags that our application will analyze:

- **Sentiment**: Identifies the emotional tone of the text (e.g., positive, negative, neutral).
- **Political Tendency**: Detects the political leaning expressed in the content.
- **Language**: Determines the language in which the text is written.

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from  langchain.output_parsers import StructuredOutputParser 

template = """"

Extract the desired information from the following passage.

only eextract the propertes mentioned  in the "Classification" function

Passage:
{input}
"""

prompt = ChatPromptTemplate.from_template(template=template)

output_parser = StrOutputParser()

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field
class Classification(BaseModel):
    
    sentimennt: str = Field(description="The sentiment of the text")
    
    political_tendency : str = Field(description= "The political tendency of the user")
    
    language : str = Field(description="The language the text is written in")

In [16]:
tagging_chain = prompt | llm.with_structured_output(Classification)

In [20]:
trump_follower_0 = """
onald Trump economic policies led to significant growth in the stock market and job creation during his presidency. 
Many Americans appreciated his "America First" approach and his ability to negotiate international trade deals
"""

trump_follower_1 = """
Donald Trump presidency was marked by constant controversy, misinformation, and a divisive political climate. 
His handling of the COVID-19 pandemic and frequent attacks on democratic institutions drew widespread criticism.
"""

trump_follower_2 = """
Donald Trump served as the 45th President of the United States from 2017 to 2021. 
He was previously a businessman and television personality before entering politics.
"""

In [22]:
tagging_chain.invoke({"input": trump_follower_2})

Classification(sentimennt='Neutral', political_tendency='Republican', language='English')

## 🧩 Schema Definition & Model Control

Defining a clear and precise schema allows us to have better control over the model’s output.

### ✅ Benefits of Schema Definition:
- **Specify allowed values** for each property.
- **Provide clear descriptions** to ensure the model correctly interprets each property.
- **Enforce required fields** that must be returned by the model.

### 🔁 Next Step
We will now redefine our Pydantic model to incorporate these controls using **enums**.


In [23]:
class Classification(BaseModel):
    
    sentimennt: str = Field(..., enum=["happy", "neutral", "sad"])
    
    political_tendency : str = Field(
        ...,
        description= "The political tendency of the user",
        enum = ["conservative", "liberal",  "independent"],

        )
    
    language : str = Field(..., enum=["spanish", "english"])

In [24]:
template = """"

Extract the desired information from the following passage.

only eextract the propertes mentioned  in the "Classification" function

Passage:
{input}
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [25]:
tagging_chain = prompt | llm.with_structured_output(Classification)

In [26]:
tagging_chain.invoke({"input" : trump_follower_1})

Classification(sentimennt='sad', political_tendency='conservative', language='english')